In [2]:
import pandas
import matplotlib as mpl
import xarray as xr
import numpy as np
import datetime as dt
dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
dir_cmc_clim='F:/data/sst/cmc/CMC0.2deg/v2/climatology/'

In [ ]:
#make cmc monthly average ssts
for lyr in range(1992,2017): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        init = 0
        for idyjl in range(1,366):
            d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
            if d.month!=imon:
                continue
            syr=str(d.year).zfill(4)
            smon=str(d.month).zfill(2)
            sdym=str(d.day).zfill(2)
            sjdy=str(idyjl).zfill(3)
            fname_tem=syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
            cmc_filename = dir_cmc + syr + '/' + sjdy + '/' + fname_tem
            #print(cmc_filename)
            ds = xr.open_dataset(cmc_filename,drop_variables=['analysis_error','sea_ice_fraction'])
            ds_masked = ds.where(ds['mask'] == 1.) 
            ds.close()
            #ds_masked = ds_masked.fillna(0)
            ds_masked['sq_sst']=ds_masked.analysed_sst**2
            if init==0:
                ds_sum = ds_masked
                init = 1
            else:
                ds_sum = xr.concat([ds_sum,ds_masked],dim = 'time')
            print(idyjl,ds_sum.dims)
        ds_sum = ds_sum.mean('time',skipna=True)
        ds_mnth.append(ds_sum)
    combined = xr.concat(ds_mnth, dim='time')
    fname_tem='monthly/monthly_average_' + syr + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + syr + fname_tem
    combined.to_netcdf(cmc_filename_out)
    

In [4]:
#make cmc climatology from monthly files
for lyr in range(1992,2018): 
    fname_tem='monthly/' + syr + 'monthly_average_' + str(lyr) + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + fname_tem
    ds = xr.open_dataset(cmc_filename_out)
    if init==0:
        ds_sum = ds
        init = 1
    else:
        ds_sum = xr.concat([ds_sum,ds],dim = 'time')
    print(idyjl,ds_sum.dims)
ds_sum = ds_sum.mean('time',skipna=True)
fname_tem='monthly_climatology_1992_2017_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
cmc_filename_out = dir_cmc + syr + fname_tem
combined.to_netcdf(cmc_filename_out)

    

NameError: name 'syr' is not defined

In [ ]:
ds2